In [1]:
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict

In [2]:
df = pd.read_feather('data/feather/FRS_simulations_with_sex_and_lung_cancer.feather')

In [3]:
ds = Dataset.from_pandas(df)

In [4]:
complex_cols = list(ds.features.keys())[:-2]

def transform(complex_sample):
    data = []
    for colname in complex_cols:
        val = complex(complex_sample[colname])
        data.append([val.real, val.imag, np.abs(val), np.angle(val)])
    return {'inputs': data}

ds = ds.map(transform, remove_columns=complex_cols)
ds = ds.rename_columns({'Label': 'luca_labels', 'sex_numeric': 'sex_labels'})
ds = ds.with_format('torch')

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [5]:
split1 = ds.train_test_split(.2)
split2 = split1['test'].train_test_split(.5)
dsd = DatasetDict({
    'train': split1['train'],
    'dev': split2['train'],
    'test': split2['test']
})

In [6]:
dsd

DatasetDict({
    train: Dataset({
        features: ['luca_labels', 'sex_labels', 'inputs'],
        num_rows: 32000
    })
    dev: Dataset({
        features: ['luca_labels', 'sex_labels', 'inputs'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['luca_labels', 'sex_labels', 'inputs'],
        num_rows: 4000
    })
})

In [7]:
dsd.save_to_disk('data/dataset')

Saving the dataset (0/2 shards):   0%|          | 0/32000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]